In [1]:
import numpy as np
import scipy.stats as stats

def calculate_confidence_interval(sample, distribution='t', confidence_level=0.95, std_general=None):
    """
    Функція для розрахунку довірчого інтервалу для середнього значення вибірки 
    з використанням t-розподілу чи нормального розподілу.

    Параметри:
    sample (array-like): Вибірка для аналізу.

    distribution (str, optional): Тип розподілу для використання у розрахунках. 
    Може бути 't' для t-розподілу Стьюдента чи 'z' для нормального розподілу. За замовчуванням 't'.

    confidence_level (float, optional): Рівень довіри для інтервалу. За замовчуванням 0.95.

    std_general (float, optional): Відоме стандартне відхилення генеральної сукупності.

    Повертає:
    tuple: Довірчий інтервал у формі (нижня межа, верхня межа).

    Використання розподілів:
    - 't' (t-розподіл Стьюдента) рекомендується використовувати, 
    коли розмір вибірки є малим (зазвичай менше 30) чи коли стандартне відхилення генеральної сукупності невідоме.

    - 'z' (нормальний розподіл) рекомендується використовувати для великих вибірок (30 чи більше), 
    де застосовна центральна гранична теорема, що дозволяє припустити, що розподіл середніх значень вибірки є нормальним.
    """
    mean = np.mean(sample)
    std_dev = np.std(sample, ddof=1) if std_general is None else std_general
    n = len(sample)
    alpha = 1 - confidence_level

    if distribution == 't':
        degrees_of_freedom = n - 1
        critical_value = stats.t.ppf(1 - alpha / 2, degrees_of_freedom)
    elif distribution == 'z':
        critical_value = stats.norm.ppf(1 - alpha / 2)
    else:
        raise ValueError("Невідомий тип розподілу. Використовуйте 't' або 'z'.")

    margin_of_error = critical_value * (std_dev / np.sqrt(n))
    confidence_interval = (mean - margin_of_error, mean + margin_of_error)

    return confidence_interval

Припустимо, ви аналізуєте дані про тривалість сну у вибірці з 50 людей. Ваша вибірка має середню тривалість сну 7.5 годин і стандартне відхилення 1.2 години.

In [2]:
np.random.seed(42)
sample = np.random.normal(loc=7.5, scale=1.2, size=50)

In [3]:
confidence_interval = calculate_confidence_interval(sample, distribution='t', confidence_level=0.95, std_general=1.2)
print(f"Довірчий інтервал для середньої: {confidence_interval}")

Довірчий інтервал для середньої: (6.888395087097756, 7.570467540287508)


Отже, 95% довірчий інтервал для тривалості сну буде від 6.88 до 7.57 годин. Це означає, що з ймовірністю 95% справжня середня тривалість сну у популяції буде в межах цього інтервалу.